In [1]:
from pyspark.sql import SparkSession, functions as F

# Cell to create a spark session
spark = (
    SparkSession.builder.appName("MAST30034 ASSIGNMENT 1 DUSTIN")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/08/20 15:18:17 WARN Utils: Your hostname, DESKTOP-3ADPNV0 resolves to a loopback address: 127.0.1.1; using 172.25.27.161 instead (on interface eth0)
22/08/20 15:18:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/20 15:18:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/20 15:18:21 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/08/20 15:18:21 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
from pyspark.sql.functions import *


In [3]:
yellow_spark = spark.read.parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/yellow_credit')

In [4]:
yellow_spark.limit(10)

tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID,fare_amount,tip_amount,total_amount,day_of_week,is_weekend,trip_length,Date,Start(ET),Attendance,Win,margin_victory/loss
2018-12-25 00:39:10,2018-12-25 01:04:33,2.56,162,263,16.5,4.58,22.88,Tue,false,25.3833,2018-12-25,12:00,19812,No,-14
2018-12-25 09:08:51,2018-12-25 09:13:27,0.9,163,162,5.0,1.45,7.25,Tue,false,4.6,2018-12-25,12:00,19812,No,-14
2018-12-25 10:29:03,2018-12-25 10:31:40,0.63,100,48,4.0,0.96,5.76,Tue,false,2.6167,2018-12-25,12:00,19812,No,-14
2018-12-25 10:58:19,2018-12-25 11:14:08,8.08,161,13,23.5,0.0,24.3,Tue,false,15.8167,2018-12-25,12:00,19812,No,-14
2018-12-25 15:52:23,2018-12-25 16:07:03,2.13,162,142,11.5,1.0,13.3,Tue,false,14.6667,2018-12-25,12:00,19812,No,-14
2018-12-25 00:31:15,2018-12-25 00:42:04,2.88,230,236,11.0,2.36,14.16,Tue,false,10.8167,2018-12-25,12:00,19812,No,-14
2018-12-25 00:51:53,2018-12-25 00:58:03,0.98,161,229,6.0,1.36,8.16,Tue,false,6.1667,2018-12-25,12:00,19812,No,-14
2018-12-25 03:53:56,2018-12-25 04:06:21,1.64,246,107,9.5,0.0,10.3,Tue,false,12.4167,2018-12-25,12:00,19812,No,-14
2018-12-25 17:03:49,2018-12-25 17:07:48,0.83,237,141,5.0,2.0,7.8,Tue,false,3.9833,2018-12-25,12:00,19812,No,-14
2018-12-25 17:38:54,2018-12-25 17:42:43,1.1,237,236,5.5,1.58,7.88,Tue,false,3.8167,2018-12-25,12:00,19812,No,-14


In [5]:
yellow_weekday = yellow_spark.filter( F.col("is_weekend") == True )
yellow_weekend = yellow_spark.filter( F.col("is_weekend") == False)

yellow_win = yellow_spark.filter( F.col("win") == "Yes" )
yellow_lose = yellow_spark.filter( F.col("win") == "No" )



In [6]:
yellow_win.count()

1330808

In [7]:
yellow_lose.count()

5085064

In [8]:
# average tip amount per hour in a weekday
aggregate_weekday = (yellow_weekday
     .groupBy(hour("tpep_pickup_datetime").alias("hour"))
     .agg(round(mean("tip_amount"),4).alias("mean_tip"))
     .orderBy("hour")
    )
aggregate_weekday.write.mode('overwrite').parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/aggregate_weekday')

In [9]:
# average tip amount per hour in a weekend

aggregate_weekend = (yellow_weekend
     .groupBy(hour("tpep_pickup_datetime").alias("hour"))
     .agg(round(mean("tip_amount"),4).alias("mean_tip"))
     .orderBy("hour")
    )

aggregate_weekend.write.mode('overwrite').parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/aggregate_weekend')

In [10]:
# average tip amount per hour when the knicks win

aggregate_win = (yellow_win
     .groupBy(hour("tpep_pickup_datetime").alias("hour"))
     .agg(round(mean("tip_amount"),4).alias("mean_tip"))
     .orderBy("hour")
    )

aggregate_win.write.mode('overwrite').parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/aggregate_win')

In [11]:
# average tip amount per hour when the knicks lose

aggregate_lose = (yellow_lose
     .groupBy(hour("tpep_pickup_datetime").alias("hour"))
     .agg(round(mean("tip_amount"),4).alias("mean_tip"))
     .orderBy("hour")
    )

aggregate_lose.write.mode('overwrite').parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/aggregate_lose')

In [12]:
# average tip amount based on when the game starts

aggregate_start = (yellow_spark
                       .groupBy(F.col("Start(ET)").alias("start_time"))
                       .agg(round(mean("tip_amount"),4).alias("mean_tip"))
                       .orderBy("start_time"))
aggregate_start.write.mode('overwrite').parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/aggregate_start')

In [13]:
# aggregate by Pickup_location

aggregate_PU = (yellow_spark
                       .groupBy(F.col("PULocationID").alias("Pickup_location"))
                       .agg(round(mean("tip_amount"),4).alias("mean_tip"))
                       .orderBy("Pickup_location"))
aggregate_PU.write.mode('overwrite').parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/aggregate_PU')

In [14]:
# Aggregate by dropoff location

aggregate_DO = (yellow_spark
                       .groupBy(F.col("DOLocationID").alias("Dropoff_location"))
                       .agg(round(mean("tip_amount"),4).alias("mean_tip"))
                       .orderBy("Dropoff_location"))
aggregate_DO.write.mode('overwrite').parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/aggregate_DO')

In [16]:
# take 10% sample from main data

SAMPLE_SIZE = 0.10

yellow_sample = yellow_spark.sample(SAMPLE_SIZE, seed=0)

yellow_sample.write.mode('overwrite').parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/yellow_sample')
